# Plot generation to compare different models

In [ ]:
root_dir = '/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Benchmarks/ActionRecognition/MTRSAP/results'

segmentation_results = ['']
classification_results = ['model_id_65095562_on_20241019-181218','model_id_65095564_on_20241019-182401', ]